In [ ]:
import math
import numpy as np
import json


from sympy import symbols, sin, cos, pi, Point, Line, Point2D, diff, oo, sympify
from sympy.simplify import simplify
from sympy.external import import_module
from sympy.core.compatibility import range
from sympy.plotting.experimental_lambdify import  lambdify

In [ ]:
t, u = symbols('t, u')
x1 = 16*(sin(t)**3)
y1 = 13*cos(t) - 5*(cos(2*t)) -2*cos(3*t) - cos(4*t)
x2 = 16*(sin(u)**3)
y2 = 13*cos(u) - 5*(cos(2*u)) -2*cos(3*u) - cos(4*u)
g = diff(y1, t)/diff(x1, t)
h = diff(y2, u)/diff(x2, u)
p1 = Point(x1,y1)
p2 = Point(x2,y2)
l1 = Line(p1, slope=g)
l2 = Line(p2, slope=h)
# int_line = l1.intersection(l2)
int_line = [Point2D(16*sin(t)**3 + ((16*sin(t)**3 - 16*sin(u)**3)*(2*sin(t)/3 + 13/(24*sin(2*t)) - 3/(4*sin(t)) - sin(3*t)/(2*(cos(t) - cos(3*t)))) - (16*sin(t)**3 - 16*sin(u)**3)*(2*sin(t)/3 - 2*sin(u)/3 - 13/(24*sin(2*u)) + 3/(4*sin(u)) + 13/(24*sin(2*t)) - 3/(4*sin(t)) + sin(3*u)/(2*(cos(u) - cos(3*u))) - sin(3*t)/(2*(cos(t) - cos(3*t)))) + 13*cos(t) - 5*cos(2*t) - 2*cos(3*t) - cos(4*t) - 13*cos(u) + 5*cos(2*u) + 2*cos(3*u) + cos(4*u))/(2*sin(t)/3 - 2*sin(u)/3 - 13/(24*sin(2*u)) + 3/(4*sin(u)) + 13/(24*sin(2*t)) - 3/(4*sin(t)) + sin(3*u)/(2*(cos(u) - cos(3*u))) - sin(3*t)/(2*(cos(t) - cos(3*t)))), -(2*sin(t)/3 + 13/(24*sin(2*t)) - 3/(4*sin(t)) - sin(3*t)/(2*(cos(t) - cos(3*t))))*((16*sin(t)**3 - 16*sin(u)**3)*(2*sin(t)/3 + 13/(24*sin(2*t)) - 3/(4*sin(t)) - sin(3*t)/(2*(cos(t) - cos(3*t)))) - (16*sin(t)**3 - 16*sin(u)**3)*(2*sin(t)/3 - 2*sin(u)/3 - 13/(24*sin(2*u)) + 3/(4*sin(u)) + 13/(24*sin(2*t)) - 3/(4*sin(t)) + sin(3*u)/(2*(cos(u) - cos(3*u))) - sin(3*t)/(2*(cos(t) - cos(3*t)))) + 13*cos(t) - 5*cos(2*t) - 2*cos(3*t) - cos(4*t) - 13*cos(u) + 5*cos(2*u) + 2*cos(3*u) + cos(4*u))/(2*sin(t)/3 - 2*sin(u)/3 - 13/(24*sin(2*u)) + 3/(4*sin(u)) + 13/(24*sin(2*t)) - 3/(4*sin(t)) + sin(3*u)/(2*(cos(u) - cos(3*u))) - sin(3*t)/(2*(cos(t) - cos(3*t)))) + 13*cos(t) - 5*cos(2*t) - 2*cos(3*t) - cos(4*t))]
int_point = int_line[0]


In [ ]:
class MyParametricLine(Parametric2DLineSeries):
    def get_segments(self):
        """
        Adaptively gets segments for plotting.

        The adaptive sampling is done by recursively checking if three
        points are almost collinear. If they are not collinear, then more
        points are added between those points.

        References
        ==========
        [1] Adaptive polygonal approximation of parametric curves,
            Luiz Henrique de Figueiredo.

        """
        if not self.adaptive:
            return super(Parametric2DLineSeries, self).get_segments()

        f_x = lambdify([self.var], self.expr_x)
        f_y = lambdify([self.var], self.expr_y)
        list_segments = []
        point_list = []

        def sample(param_p, param_q, p, q, depth):
            """ Samples recursively if three points are almost collinear.
            For depth < 6, points are added irrespective of whether they
            satisfy the collinearity condition or not. The maximum depth
            allowed is 12.
            """
            #Randomly sample to avoid aliasing.
            np = import_module('numpy')
            random = 0.45 + np.random.rand() * 0.1
            param_new = param_p + random * (param_q - param_p)
            xnew = f_x(param_new)
            ynew = f_y(param_new)
            new_point = np.array([xnew, ynew])

            #Maximum depth
            if depth > self.depth:
                list_segments.append([p, q])
                point_list.append(param_p)
                
            #Sample irrespective of whether the line is flat till the
            #depth of 6. We are not using linspace to avoid aliasing.
            elif depth < 6:
                sample(param_p, param_new, p, new_point, depth + 1)
                sample(param_new, param_q, new_point, q, depth + 1)

            #Sample ten points if complex values are encountered
            #at both ends. If there is a real value in between, then
            #sample those points further.
            elif ((p[0] is None and q[1] is None) or
                    (p[1] is None and q[1] is None)):
                param_array = np.linspace(param_p, param_q, 10)
                x_array = list(map(f_x, param_array))
                y_array = list(map(f_y, param_array))
                if any(x is not None and y is not None
                        for x, y in zip(x_array, y_array)):
                    for i in range(len(y_array) - 1):
                        if ((x_array[i] is not None and y_array[i] is not None) or
                                (x_array[i + 1] is not None and y_array[i + 1] is not None)):
                            point_a = [x_array[i], y_array[i]]
                            point_b = [x_array[i + 1], y_array[i + 1]]
                            sample(param_array[i], param_array[i], point_a,
                                   point_b, depth + 1)

            #Sample further if one of the end points in None( ie a complex
            #value) or the three points are not almost collinear.
            elif (p[0] is None or p[1] is None
                    or q[1] is None or q[0] is None
                    or not flat(p, new_point, q)):
                sample(param_p, param_new, p, new_point, depth + 1)
                sample(param_new, param_q, new_point, q, depth + 1)
            else:
                list_segments.append([p, q])
                point_list.append(param_p)

        f_start_x = f_x(self.start)
        f_start_y = f_y(self.start)
        start = [f_start_x, f_start_y]
        f_end_x = f_x(self.end)
        f_end_y = f_y(self.end)
        end = [f_end_x, f_end_y]
        sample(self.start, self.end, start, end, 0)
        return list_segments, point_list

In [ ]:
def get_intersection_points(min_t, max_t, int_point, steps=100,):
    """
    num_cycles: 
        how many times does the spiral go around (this will be rounded up to give a complete segment)
    angle_change:
        how large is the change in angle for each quadratic segment
    """
#     t_range = np.linspace(min_t, max_t, num=steps, endpoint=True)
    flahp = MyParametricLine(x1,y1, (t,min_t,max_t), nb_of_points=steps)
    t_range = flahp.get_segments()[1]


    for i, t2 in enumerate(t_range[:-1]):
        u = t_range[i+1]
        try:             
            yield {"int_point": int_point.subs({"t":t2, "u":u}),
                   "t": t,
                   "u": u,
                  }
        except ValueError:
            t3 = t2+0.0000001
            yield {"int_point": int_point.subs({"t":t3, "u":u}),
                       "t": t,
                       "u": u,
                      }

In [ ]:
def get_quad_control_points(min_t, max_t, int_point, steps=100):
    for x in get_intersection_points(min_t, max_t, int_point, steps=steps):
        yield (tuple(map(float, x['int_point'].args)), 
               tuple(map(float, p2.subs({'u':x['u']}).args))
              )

In [ ]:
def store_data(fname, quad_points, initial_point):
    data = {"quad_points": quad_points,
     "initial_point": initial_point
    }
    with open(fname, "w") as fp:
        json.dump(data, fp)

def load_data(fname):
    with open(fname, "r") as fp:
        return json.load(fp)

In [ ]:
t_0 = 0
t_max = 2*pi
quad_iter = get_quad_control_points(min_t=t_0, max_t=t_max, steps=10)
quad_points = list(quad_iter)
initial_point = list(map(float, p1.subs({"t":t_0}).args))

In [ ]:
store_data('heart_quads.json', quad_points, initial_point)

In [1]:
import json
import math

import numpy as np

import cairosvg

from uuid import uuid4

from vdom.svg import *
from vdom.svg import g as group

from IPython.display import display_png

In [3]:
data = load_data('heart_quads.json')
quads = data['quad_points']
initial_point = data['initial_point']
center = 0

d = f'M{initial_point[0]},{initial_point[1]} {" ".join("Q"+",".join(map(str, (q for q in quad[0])))+" "+",".join(map(str, (q for q in quad[1]))) for quad in quads)} '
def gen_id():
    return f"{uuid4()}".replace("-","")

myid = gen_id()

In [4]:
def gen_heart(myid=None, stroke="red"):
    if myid is None:
        myid = gen_id()
    d = f'M{initial_point[0]},{initial_point[1]} {" ".join("Q"+",".join(map(str, (q for q in quad[0])))+" "+",".join(map(str, (q for q in quad[1]))) for quad in quads)} Z'
    my_heart = path(id=f"{myid}",
                    fill="none", 
                    stroke=stroke,
                    d=d,
                    transform = "scale(1.25 -1.25)",
                    **{"stroke-width": ".5"}
        )    
    return my_heart


In [5]:
def grid():    
    return group(
        line(x1="-50%", y1="0%", x2="50%", y2="0%", stroke="black", **{"stroke-width":".25"}),
        line(x1="0%", y1="-50%", x2="0%", y2="50%", stroke="black", **{"stroke-width":".25"}),
    )


In [7]:
a_heart = svg(gen_heart(),
              grid(),
              viewBox="-25 -25 50 50",)
display(a_heart)

<svg viewBox="-25 -25 50 50"><path d="M0.0,5.0 Q2.26480878769362e-09,5.01935203545116 0.00240468499265089,5.05784209373124 Q0.00617678614768245,5.11821945317385 0.0144892681784808,5.19071052417724 Q0.0249796452381706,5.28219447150715 0.0416669391001586,5.38322902221801 Q0.0625689845764014,5.50978212250472 0.0921242540272451,5.64469196933818 Q0.129325564905271,5.81450341906648 0.178888071354544,5.99135494886849 Q0.227032997926876,6.16314819595208 0.285852572783466,6.33796304924934 Q0.345659522697073,6.51571243286517 0.415701700411068,6.69417844133358 Q0.487743067528425,6.87773834271958 0.569888188951648,7.06019887319993 Q0.692043825721294,7.33153065651206 0.834806300015498,7.59665683949396 Q0.918421740467044,7.75194024064588 1.00862101667763,7.90415568501767 Q1.12167926629051,8.09494670272544 1.24449884261148,8.27994312284276 Q1.44189780344349,8.57727439675439 1.66250874377508,8.85670281234556 Q1.80054060640138,9.03153559541965 1.94694688614682,9.19842404729501 Q2.08999423977703,9.36148366081515 2.24054566891579,9.51639833059041 Q2.48490085256662,9.76783534981824 2.74741879975013,9.99621086079807 Q3.01662696842382,10.2304064795754 3.30291999709097,10.4385942042024 Q3.46972317871875,10.5598908176942 3.64179542494331,10.6719572214367 Q3.79857226878001,10.7740621074039 3.95941554482475,10.8683033405894 Q4.13334243392154,10.9702105195257 4.31166833497306,11.0627149706127 Q4.47604708828145,11.1479845038409 4.64386151028959,11.2251070150907 Q4.80544594869858,11.299366411392 4.96995032351617,11.3659501831024 Q5.16725965014621,11.4458118838945 5.36835311845226,11.5144650549576 Q5.56561833961004,11.5818112650908 5.7660857806164,11.6382214352365 Q5.93594304380552,11.6860181101922 6.10780540394526,11.7258795037075 Q6.3802811445957,11.7890769525997 6.65689783599178,11.8321328272072 Q6.88756600251257,11.8680367330906 7.12039035026157,11.8898178268749 Q7.33575429714658,11.9099654719568 7.5524023719199,11.9179755377665 Q7.78903411552322,11.9267244539104 8.02650188595541,11.9209696911957 Q8.31148267851657,11.9140635041521 8.59647282523938,11.8863060110558 Q8.84172783653293,11.8624186438571 9.08605624824551,11.8231757557194 Q9.29602775102361,11.7894511138005 9.5047004646898,11.7444820539865 Q9.73546771599974,11.6947516117789 9.96389320433066,11.631434414029 Q10.2137994798527,11.5621629628359 10.4598847954114,11.4769149350853 Q10.7426952738113,11.3789447028671 11.0189146985215,11.2604175545564 Q11.2783606307036,11.14908799649 11.5305552606758,11.0202359184706 Q11.8305962880828,10.8669380077389 12.1182124982183,10.6899608334626 Q12.3086605446971,10.5727735681955 12.4928101128895,10.4457122376404 Q12.7138340887796,10.2932079405257 12.9247407405014,10.1272078034175 Q13.1164494580624,9.97631795172166 13.2989309836935,9.81495653041487 Q13.4993501715355,9.63773352169933 13.6876496499027,9.44875874377583 Q13.8259025844505,9.31000999189398 13.9571663998683,9.16537567684822 Q14.0895559130142,9.01950100078341 14.2144709921875,8.86804150073268 Q14.3647042289743,8.68588374135466 14.50359102542,8.496302620215 Q14.6242491373507,8.33160373796121 14.7359701511584,8.16180777241553 Q14.8556947497797,7.9798477889744 14.9647911861361,7.79259020255814 Q15.0815966885041,7.5921004598673 15.1858302302757,7.38620717426887 Q15.2838990649354,7.19249108020058 15.3705462405702,6.99456656547992 Q15.4710479022511,6.76499482847765 15.555848418953,6.53053381443338 Q15.6293410875024,6.32733734508333 15.6908398354807,6.12101489646327 Q15.7582996840502,5.89469352248478 15.811151998362,5.66520400901477 Q15.858001700426,5.46177837907041 15.8932743283624,5.25628456535619 Q15.9234569772515,5.08044431780161 15.9451194504346,4.90333416558016 Q15.9730560934491,4.67492704627213 15.9867773968648,4.44479362169134 Q16.0034165941822,4.16572133934909 15.9991176647554,3.88472444785707 Q15.9950908430473,3.62151371089842 15.9726972179107,3.35708697651958 Q15.9526619394994,3.12050789606779 15.9179459679384,2.88322942110747 Q15.882312030686,2.63967678789829 15.83124032874,2.39559405324708 Q15.78751